In [4]:
# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.
This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.
This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
from config import yelp_key

In [13]:
# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

<function urllib.parse.quote(string, safe='/', encoding=None, errors=None)>

In [6]:
# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY=yelp_key

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3
# print(yelp_key)


In [9]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [21]:
# url_params={"location":"Seattle"}

url_params={"city":"St. Louis County", 'zip_code':'63131','latitude':'38.618582','longitude':'-90.43643'}
request(API_HOST, SEARCH_PATH, API_KEY, url_params)

Querying https://api.yelp.com/v3/businesses/search ...


{'businesses': [{'id': 'IO3apWfFnKPO2XRFmGfTEw',
   'alias': 'nathaniel-reid-bakery-saint-louis',
   'name': 'Nathaniel Reid Bakery',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/grdD_OnUFxWSJ6Buu4Ecvg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/nathaniel-reid-bakery-saint-louis?adjust_creative=mtl9Z6oZg9mUuyl96J8xBg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mtl9Z6oZg9mUuyl96J8xBg',
   'review_count': 339,
   'categories': [{'alias': 'bakeries', 'title': 'Bakeries'},
    {'alias': 'desserts', 'title': 'Desserts'},
    {'alias': 'sandwiches', 'title': 'Sandwiches'}],
   'rating': 5.0,
   'coordinates': {'latitude': 38.5991673516503,
    'longitude': -90.4169680166228},
   'transactions': [],
   'price': '$',
   'location': {'address1': '11243 Manchester Rd',
    'address2': '',
    'address3': None,
    'city': 'Saint Louis',
    'zip_code': '63122',
    'country': 'US',
    'state': 'MO',
    'display_address': ['11243 Manc

In [10]:
def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [17]:
search(API_KEY,'Bricktops',"St. Louis")

Querying https://api.yelp.com/v3/businesses/search ...


{'businesses': [{'id': 'd3dw3YihB3WuU1Sb_i74-w',
   'alias': 'bricktops-frontenac',
   'name': "BrickTop's",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/QP5gFHPXbnAdQUnKXvNN4Q/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/bricktops-frontenac?adjust_creative=mtl9Z6oZg9mUuyl96J8xBg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=mtl9Z6oZg9mUuyl96J8xBg',
   'review_count': 216,
   'categories': [{'alias': 'newamerican', 'title': 'American (New)'}],
   'rating': 3.5,
   'coordinates': {'latitude': 38.6328651, 'longitude': -90.4078657},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '10342 Clayton Rd',
    'address2': '',
    'address3': '',
    'city': 'Frontenac',
    'zip_code': '63131',
    'country': 'US',
    'state': 'MO',
    'display_address': ['10342 Clayton Rd', 'Frontenac, MO 63131']},
   'phone': '+13145676300',
   'display_phone': '(314) 567-6300',
   'distance': 7824.811825747739},
  {'id': '3Ym00dQ

In [8]:
def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

In [9]:
def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [16]:
query_api("Diner", "St. Louis")

Querying https://api.yelp.com/v3/businesses/search ...
3 businesses found, querying business info for the top result "k8hFz4Gty5dHlDV4Wt8FqA" ...
Querying https://api.yelp.com/v3/businesses/k8hFz4Gty5dHlDV4Wt8FqA ...
Result for business "k8hFz4Gty5dHlDV4Wt8FqA" found:
{ 'alias': 'southwest-diner-saint-louis',
  'categories': [{'alias': 'diners', 'title': 'Diners'}],
  'coordinates': {'latitude': 38.61342, 'longitude': -90.30569},
  'display_phone': '(314) 260-7244',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '1400',
                           'is_overnight': False,
                           'start': '0700'},
                         { 'day': 1,
                           'end': '1400',
                           'is_overnight': False,
                           'start': '0700'},
                         { 'day': 2,
                           'end': '1400',
                         

In [20]:
def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
                        type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location',
                        default=DEFAULT_LOCATION, type=str,
                        help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except HTTPError as error:
        sys.exit(
            'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                error.code,
                error.url,
                error.read(),
            )
        )


# if __name__ == '__main__':
#     main()